<h1 align = "center"><font>BEST MOVIE ANALYZER. ROAD TO THE MODEL</font></h1>

In [2]:
#All libs we are required (while project is not finished the list will become more biger!):
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier

<h2 align = "center"><font>FIRST LOOK AT DATA</font></h2>
<h3 align = "center"><font color='#4e78f5'>Our dataset in general:</font></h3>

In [16]:
#First look at head of dataset:
movie_df_all = pd.read_csv('movieset_6800.csv')
movie_df_all.shape

(7668, 15)

In [17]:
movie_df_all.head(5)

,name,rating,genre,year,released,score,votes,director,writer,star,country,budget,gross,company,runtime
0,The Shining,R,Drama,1980,"June 13, 1980 (United States)",8.4,927000.0,Stanley Kubrick,Stephen King,Jack Nicholson,United Kingdom,19000000.0,46998772.0,Warner Bros.,146.0
1,The Blue Lagoon,R,Adventure,1980,"July 2, 1980 (United States)",5.8,65000.0,Randal Kleiser,Henry De Vere Stacpoole,Brooke Shields,United States,4500000.0,58853106.0,Columbia Pictures,104.0
2,Star Wars: Episode V - The Empire Strikes Back,PG,Action,1980,"June 20, 1980 (United States)",8.7,1200000.0,Irvin Kershner,Leigh Brackett,Mark Hamill,United States,18000000.0,538375067.0,Lucasfilm,124.0
3,Airplane!,PG,Comedy,1980,"July 2, 1980 (United States)",7.7,221000.0,Jim Abrahams,Jim Abrahams,Robert Hays,United States,3500000.0,83453539.0,Paramount Pictures,88.0
4,Caddyshack,R,Comedy,1980,"July 25, 1980 (United States)",7.3,108000.0,Harold Ramis,Brian Doyle-Murray,Chevy Chase,United States,6000000.0,39846344.0,Orion Pictures,98.0


<h3 align = "center"><font color='#4e78f5'>General situation:</font></h3>

<p>We have 7668 movies and different inforamtion about this movies.</p>
<p>Name - name of the movie.</p>
<p>Rating - who can watch this movie.</p>
<p>Genre - groups of feature films, distinguished on the basis of similar features of their internal structure.</p>
<p>Year - released year.</p>
<p>Released - released date with additional information.</p>
<p>Score - IMDB score.</p>
<p>Votes - number of votes.</p>
<p>Director - director who was produced film.</p>
<p>Writer - person who wrote history.</p>
<p>Star - the main actor of the movie.</p>
<p>Budget - quntity of money spent to the film.</p>
<p>Gross - quantity of money movie earned.</p>
<p>Company - company that produced the movie.</p>
<p>Runtime - duration of the movie</p>

<h2 align = "center"><font>RATE FEATURE</font></h2>
<h3 align = "center"><font color='#6ad156'>One-hot-encoding rate feature:</font></h3>

In [18]:
#Sepparte columns with rate
movie_df_ohe = pd.get_dummies(movie_df_all, columns = ['rating'])
movie_df_ohe

,name,genre,year,released,score,votes,director,writer,star,country,...,rating_NC-17,rating_Not Rated,rating_PG,rating_PG-13,rating_R,rating_TV-14,rating_TV-MA,rating_TV-PG,rating_Unrated,rating_X
0,The Shining,Drama,1980,"June 13, 1980 (United States)",8.4,927000.0,Stanley Kubrick,Stephen King,Jack Nicholson,United Kingdom,...,0,0,0,0,1,0,0,0,0,0
1,The Blue Lagoon,Adventure,1980,"July 2, 1980 (United States)",5.8,65000.0,Randal Kleiser,Henry De Vere Stacpoole,Brooke Shields,United States,...,0,0,0,0,1,0,0,0,0,0
2,Star Wars: Episode V - The Empire Strikes Back,Action,1980,"June 20, 1980 (United States)",8.7,1200000.0,Irvin Kershner,Leigh Brackett,Mark Hamill,United States,...,0,0,1,0,0,0,0,0,0,0
3,Airplane!,Comedy,1980,"July 2, 1980 (United States)",7.7,221000.0,Jim Abrahams,Jim Abrahams,Robert Hays,United States,...,0,0,1,0,0,0,0,0,0,0
4,Caddyshack,Comedy,1980,"July 25, 1980 (United States)",7.3,108000.0,Harold Ramis,Brian Doyle-Murray,Chevy Chase,United States,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7663,More to Life,Drama,2020,"October 23, 2020 (United States)",3.1,18.0,Joseph Ebanks,Joseph Ebanks,Shannon Bond,United States,...,0,0,0,0,0,0,0,0,0,0
7664,Dream Round,Comedy,2020,"February 7, 2020 (United States)",4.7,36.0,Dusty Dukatz,Lisa Huston,Michael Saquella,United States,...,0,0,0,0,0,0,0,0,0,0
7665,Saving Mbango,Drama,2020,"April 27, 2020 (Cameroon)",5.7,29.0,Nkanya Nkwai,Lynno Lovert,Onyama Laura,United States,...,0,0,0,0,0,0,0,0,0,0
7666,It's Just Us,Drama,2020,"October 1, 2020 (United States)",NaN,NaN,James Randall,James Randall,Christina Roz,United States,...,0,0,0,0,0,0,0,0,0,0


In [19]:
#Create my rate system where M_A - middle audience. U_A - universal audience. A_A - adult audeince

#M_A
movie_df_ohe["Rate_M_A"] = (movie_df_ohe["rating_PG"] + movie_df_ohe["rating_PG-13"] + 
movie_df_ohe["rating_TV-PG"])

#U_A
movie_df_ohe["Rate_U_A"] = (movie_df_ohe["rating_G"] + movie_df_ohe["rating_TV-14"] + movie_df_ohe["rating_Approved"])

#A_A
movie_df_ohe["Rate_A_A"] = (movie_df_ohe["rating_NC-17"] + movie_df_ohe["rating_Not Rated"] + movie_df_ohe["rating_R"]
+ movie_df_ohe["rating_TV-MA"] + movie_df_ohe["rating_Unrated"] + movie_df_ohe["rating_X"])

#Delete useless columns
movie_df_ohe = movie_df_ohe.drop(columns = ["rating_PG", "rating_PG-13", "rating_TV-PG",
 "rating_TV-14", "rating_NC-17", "rating_Not Rated", "rating_R", "rating_TV-MA", "rating_Unrated", "rating_X", "rating_G", "rating_Approved"])

<h3 align = "center"><font color='#6ad156'>Rate feature correct for model learning:</font></h3>

In [20]:
#Current DataSet:
movie_df_ohe[["Rate_A_A", "Rate_M_A", "Rate_U_A"]]

,Rate_A_A,Rate_M_A,Rate_U_A
0,1,0,0
1,1,0,0
2,0,1,0
3,0,1,0
4,1,0,0
...,...,...,...
7663,0,0,0
7664,0,0,0
7665,0,0,0
7666,0,0,0


In [21]:
#General information about or new columns:
print("U_A count:", movie_df_ohe['Rate_U_A'].sum())

print("A_A count:", movie_df_ohe['Rate_A_A'].sum())

print("M_A count:", movie_df_ohe['Rate_M_A'].sum())

print("Nan count:", len(movie_df_ohe) - 
(movie_df_ohe['Rate_U_A'].sum() + movie_df_ohe['Rate_A_A'].sum() + movie_df_ohe['Rate_M_A'].sum()))

U_A count: 155
A_A count: 4067
M_A count: 3369
Nan count: 77


<h2 align = "center"><font>GENRE FEATURE</font></h2>
<h3 align = "center"><font color='#2db3c2'>One-hot-encoding genre feature</font></h3>

In [22]:
#Sepparte columns with genre due to the hot-encoding technique
#movie_df_ohe = pd.get_dummies(movie_df_all, columns = ['genre'])
movie_df_ohe

,name,rating,year,released,score,votes,director,writer,star,country,...,genre_History,genre_Horror,genre_Music,genre_Musical,genre_Mystery,genre_Romance,genre_Sci-Fi,genre_Sport,genre_Thriller,genre_Western
0,The Shining,R,1980,"June 13, 1980 (United States)",8.4,927000.0,Stanley Kubrick,Stephen King,Jack Nicholson,United Kingdom,...,0,0,0,0,0,0,0,0,0,0
1,The Blue Lagoon,R,1980,"July 2, 1980 (United States)",5.8,65000.0,Randal Kleiser,Henry De Vere Stacpoole,Brooke Shields,United States,...,0,0,0,0,0,0,0,0,0,0
2,Star Wars: Episode V - The Empire Strikes Back,PG,1980,"June 20, 1980 (United States)",8.7,1200000.0,Irvin Kershner,Leigh Brackett,Mark Hamill,United States,...,0,0,0,0,0,0,0,0,0,0
3,Airplane!,PG,1980,"July 2, 1980 (United States)",7.7,221000.0,Jim Abrahams,Jim Abrahams,Robert Hays,United States,...,0,0,0,0,0,0,0,0,0,0
4,Caddyshack,R,1980,"July 25, 1980 (United States)",7.3,108000.0,Harold Ramis,Brian Doyle-Murray,Chevy Chase,United States,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7663,More to Life,NaN,2020,"October 23, 2020 (United States)",3.1,18.0,Joseph Ebanks,Joseph Ebanks,Shannon Bond,United States,...,0,0,0,0,0,0,0,0,0,0
7664,Dream Round,NaN,2020,"February 7, 2020 (United States)",4.7,36.0,Dusty Dukatz,Lisa Huston,Michael Saquella,United States,...,0,0,0,0,0,0,0,0,0,0
7665,Saving Mbango,NaN,2020,"April 27, 2020 (Cameroon)",5.7,29.0,Nkanya Nkwai,Lynno Lovert,Onyama Laura,United States,...,0,0,0,0,0,0,0,0,0,0
7666,It's Just Us,NaN,2020,"October 1, 2020 (United States)",NaN,NaN,James Randall,James Randall,Christina Roz,United States,...,0,0,0,0,0,0,0,0,0,0
